In [ ]:
from terratorch.datasets.od_aed_elephant import ElephantCocoDataset
import socket

In [ ]:

hostname = socket.gethostname()
print(f"The hostname is: {hostname}")

# Output will be similar to:
# The hostname is: my-computer-name

In [ ]:
if socket.gethostname() == "li-dee0034c-2e23-11b2-a85c-bbf4df0b1948": # romeo
    img_folder_train='/home/romeokienzler/Downloads/AED/training_images'
    ann_file_train='/home/romeokienzler/Downloads/AED/annotations_elephants_training.json'
    img_folder_test='/home/romeokienzler/Downloads/AED/test_images'
    ann_file_test='/home/romeokienzler/Downloads/AED/annotations_elephants_test.json'
else:
    img_folder_train='/dccstor/terratorch/shared/datasets/aed-elephant/AED/training_images'
    ann_file_train='/dccstor/terratorch/shared/datasets/aed-elephant/annotations_elephants_train.json'
    img_folder_test='/dccstor/terratorch/shared/datasets/aed-elephant/AED/test_images'
    ann_file_test='/dccstor/terratorch/shared/datasets/aed-elephant/annotations_elephants_test.json'

ds = ElephantCocoDataset(
    img_folder=img_folder_train,
    ann_file=ann_file_train,
)


In [ ]:
len(ds)

In [ ]:
e = ds[0]

In [ ]:
e

In [ ]:
h, w = e['image'].shape[-2:]

In [ ]:
w

In [ ]:
from terratorch.datamodules.od_aed_elephant import ElephantDataModule
dm = ElephantDataModule(img_folder_train=img_folder_train, ann_file_train=ann_file_train, img_folder_val=img_folder_test, ann_file_val=ann_file_test, min_size=(3648, 5472), tile_size=(512,512), overlap=128, batch_size=8, num_workers=0)


  

In [ ]:
dm.setup()

In [ ]:
tdl = dm.train_dataloader()

In [ ]:
len(tdl.dataset)

In [ ]:
tdl.dataset[0].keys()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision.transforms.functional as F
import torch

def show_from_datamodule(dm, n=3, split="train"):
    """Show exactly n samples with bounding boxes from a DataModule."""
    # pick the right loader
    if split == "train":
        loader = dm.train_dataloader()
    elif split == "val":
        loader = dm.val_dataloader()
    elif split == "test":
        loader = dm.test_dataloader()
    else:
        raise ValueError("split must be 'train', 'val', or 'test'")
    
    shown = 0
    for batch in loader:  # loop until enough samples found
        images = batch["image"]     # (B, C, H, W)
        boxes  = batch["boxes"]     # list[Tensor] or padded tensor

        for img, b in zip(images, boxes):
            # skip samples with no boxes
            if b is None or (isinstance(b, torch.Tensor) and b.numel() == 0) or len(b) == 0:
                continue  

            if torch.is_tensor(img):
                img = F.to_pil_image(img)

            fig, ax = plt.subplots(1, 1, figsize=(6, 6))
            ax.imshow(img)

            # convert to tensor if needed
            if isinstance(b, torch.Tensor):
                b = b.cpu()
            for box in b:
                x1, y1, x2, y2 = box.tolist()
                rect = patches.Rectangle(
                    (x1, y1), x2 - x1, y2 - y1,
                    linewidth=2, edgecolor="red", facecolor="none"
                )
                ax.add_patch(rect)

            ax.set_title(f"Sample {shown}, {len(b)} objects")
            plt.axis("off")
            plt.show()

            shown += 1
            if shown >= n:
                return  # stop once we have shown n samples


In [ ]:
#m.setup("fit")
#show_from_datamodule(dm, n=30, split="train")

In [ ]:
""" train_loader = dm.train_dataloader()

# Iterate and count
total_elements = 0
for batch in train_loader:
    # 'batch' is a tensor or list of tensors depending on your dataset
    batch_size = len(batch)
    print(f"Processing a batch of size: {batch_size}")
    total_elements += batch_size

print(f"All elements read. Total count: {total_elements}") """


In [ ]:
from terratorch.tasks.object_detection_task import ObjectDetectionTask

model = ObjectDetectionTask.load_from_checkpoint("./best-epoch=18.ckpt")

In [ ]:
from PIL import Image
import torch
from torchvision.transforms import functional as F

# load png → tensor
path = "tile_cache/tile_101_3072_384.png"
img = Image.open(path).convert("RGB")
tensor = F.to_tensor(img).unsqueeze(0)  # shape (1, C, H, W) for model input

# send to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tensor = tensor.to(device)

# example model inference
model.eval()

with torch.no_grad():
    output = model(tensor)


In [ ]:
output

In [ ]:
def plot_detection(img: torch.Tensor, output, suptitle: str | None = None) -> plt.Figure:
    """
    Plot image with object detection predictions from ModelOutputObjectDetection.

    Args:
        img: torch.Tensor in (C,H,W) or (B,C,H,W), range [0,1]
        output: ModelOutputObjectDetection object
        suptitle: Optional title for figure
    """
    # handle batch dimension
    if img.ndim == 4:
        if img.shape[0] != 1:
            raise ValueError(f"Expected batch of size 1, got {img.shape[0]}")
        img = img[0]

    # CHW → HWC for matplotlib
    img_np = img.permute(1, 2, 0).cpu().numpy()

    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.imshow(img_np)
    ax.axis("off")

    if output is not None and output.output is not None:
        for pred in output.output:
            boxes = pred.get("boxes", [])
            labels = pred.get("labels", [])
            scores = pred.get("scores", [])

            for box, label, score in zip(boxes, labels, scores):
                x1, y1, x2, y2 = box.tolist()
                w, h = x2 - x1, y2 - y1
                rect = patches.Rectangle(
                    (x1, y1), w, h,
                    linewidth=2, edgecolor="lime", facecolor="none"
                )
                ax.add_patch(rect)
                ax.text(
                    x1, y1,
                    f"{label.item()} ({score.item():.2f})",
                    fontsize=8,
                    color="black",
                    bbox=dict(facecolor="lime", alpha=0.5, edgecolor="none", pad=1)
                )

    if suptitle is not None:
        fig.suptitle(suptitle)

    return fig


In [ ]:
plot_detection(tensor, output)


In [ ]:


from terratorch.models.object_detection_model_factory import ModelOutputObjectDetection
import torch
from torch import Tensor

class ONNXWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, *args, **kwargs):
        out = self.model(*args, **kwargs)

        # --- Handle ModelOutputObjectDetection ---
        if isinstance(out, ModelOutputObjectDetection):
            if isinstance(out.output, dict):
                # Single image
                boxes = out.output.get("boxes", torch.empty(0, 4))
                scores = out.output.get("scores", torch.empty(0))
                labels = out.output.get("labels", torch.empty(0, dtype=torch.long))
                return boxes, scores, labels

            elif isinstance(out.output, list):
                # Batch: flatten all boxes/scores/labels into single tensors
                all_boxes, all_scores, all_labels = [], [], []
                for item in out.output:
                    all_boxes.append(item.get("boxes", torch.empty(0, 4)))
                    all_scores.append(item.get("scores", torch.empty(0)))
                    all_labels.append(item.get("labels", torch.empty(0, dtype=torch.long)))

                boxes = torch.cat(all_boxes, dim=0) if all_boxes else torch.empty(0, 4)
                scores = torch.cat(all_scores, dim=0) if all_scores else torch.empty(0)
                labels = torch.cat(all_labels, dim=0) if all_labels else torch.empty(0, dtype=torch.long)
                return boxes, scores, labels

        # --- If it's already a tensor ---
        if isinstance(out, Tensor):
            return out

        # --- Fallback ---
        raise TypeError(f"Unsupported model output type: {type(out)}")



In [ ]:
onnx_model = ONNXWrapper(model)

In [ ]:
onnx_model.eval()

In [ ]:
dummy_input = torch.randn(1, 3, 512,512)  # batch size 1, 10 features

In [ ]:
torch.onnx.export(
    onnx_model,              # the PyTorch model
    dummy_input,              # dummy input
    "model_tm_tiny_aed_elephants.onnx",             # output file     
    export_params=True,       # store trained weights
    opset_version=17,         # recommended ONNX opset
    do_constant_folding=True, # optimize constants
    input_names=['input'],    # model input name
    output_names=['output'],  # model output name
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}  # allow variable batch size
)